In [124]:
import os
import sys
import geopandas as gpd
import rasterio
from osgeo import gdal, osr
import pandas as pd
import numpy as np
from pathlib import Path
import folium
import glob
from folium.raster_layers import ImageOverlay
from rasterio.plot import show
from rasterio.mask import mask
from matplotlib import pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from rasterio.features import rasterize
from shapely.geometry import shape
from folium import GeoJson



In [125]:
country = 'Dominican Republic'
output_folder = Path(r'Output')

In [126]:
aoi_gdf = gpd.read_file('AOI/Edesur.shp')
tiff_path = 'Raster/edesur_avg_rad_sum.tif'
def add_colored_tiff_layer(map_obj, tiff_path,layer_name ='Colored Tiff Path', colormap='Spectral'):
    with rasterio.open(tiff_path) as src:
        # Read the first band of the TIFF file
        data = src.read(1)
        
        # Convert zero and negative values to NaN to exclude them
        data = np.where(data <= 0, np.nan, data)
        
        # Normalize data for colormap scaling, ignoring NaNs
        norm = Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data))
        colormap_instance = ScalarMappable(norm=norm, cmap=colormap)
        
        # Apply colormap, converting NaNs to transparent
        colored_data = colormap_instance.to_rgba(data)
        
        # Set NaN values to fully transparent in the alpha channel
        colored_data[..., 3] = np.where(np.isnan(data), 0, colored_data[..., 3])
        
        # Save the color-mapped image as PNG with transparency
        output_path = 'Raster/temp_colored_image.png'
        plt.imsave(output_path, colored_data, format='png')
        
        # Define the correct bounding box for the ImageOverlay
        bounds = src.bounds
        image_bounds = [[bounds.bottom, bounds.left], [bounds.top, bounds.right]]
        
        # Add the color-mapped image as an ImageOverlay with correct bounds
        ImageOverlay(
            name= layer_name,
            image=output_path,
            bounds=image_bounds,
            opacity=0.7,
            interactive=True
        ).add_to(map_obj)



In [127]:
def add_colored_tiff_layer(map_obj, tiff_path, layer_name='Colored TIFF Layer', colormap='Spectral', vectorize=False):
    with rasterio.open(tiff_path) as src:
        # Read the first band of the TIFF file
        data = src.read(1)

        # Convert zero and negative values to NaN to exclude them
        data = np.where(data <= 0, np.nan, data)

        # Normalize data for colormap scaling, ignoring NaNs
        norm = Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data))
        colormap_instance = ScalarMappable(norm=norm, cmap=colormap)

        # Apply colormap, converting NaNs to transparent
        colored_data = colormap_instance.to_rgba(data)

        # Set NaN values to fully transparent in the alpha channel
        colored_data[..., 3] = np.where(np.isnan(data), 0, colored_data[..., 3])

        # Save the color-mapped image as PNG with transparency
        output_path = 'Raster/temp_colored_image.png'
        plt.imsave(output_path, colored_data, format='png')

        # Define the correct bounding box for the ImageOverlay
        bounds = src.bounds
        image_bounds = [[bounds.bottom, bounds.left], [bounds.top, bounds.right]]

        # Add the color-mapped image as an ImageOverlay with correct bounds
        ImageOverlay(
            name=layer_name,
            image=output_path,
            bounds=image_bounds,
            opacity=0.7,
            interactive=True
        ).add_to(map_obj)

        if vectorize:
            # Vectorize the raster data
            data[data < 0] = np.nan  # Ensure negatives are NaN
            mask = ~np.isnan(data)

            # Ensure the data type is compatible
            if data.dtype not in ['int16', 'int32', 'uint8', 'uint16', 'float32']:
                data = data.astype('float32')  # Convert to float32

            shapes = rasterio.features.shapes(data, mask=mask, transform=src.transform)
            
            # Create a list of geometries and values
            geometries = []
            values = []
            for geom, val in shapes:
                geometries.append(shape(geom))
                values.append(val)

            # Create a GeoDataFrame
            vector_gdf = gpd.GeoDataFrame({'geometry': geometries, 'value': values}, crs=src.crs)

            # Add GeoJson layer for vectorized data
            GeoJson(
                vector_gdf,
                name='Vectorized Raster Values',
                style_function=lambda feature: {
                    'color': 'black',
                    'weight': 0.5,
                    'fillColor': 'transparent',
                    'fillOpacity': 0.7
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=['value'],
                    aliases=['Value:'],
                    localize=True
                )
            ).add_to(map_obj)


In [87]:
# def rasterize_and_add_to_map(gdf, map_obj, layer_name, colormap='Blues', resolution=0.001, opacity=0.6):
#     # Define the bounds for the raster based on the vector data
#     bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]

#     # Define the transform and raster size based on the bounds and resolution
#     width = int((bounds[2] - bounds[0]) / resolution)
#     height = int((bounds[3] - bounds[1]) / resolution)
#     transform = rasterio.transform.from_bounds(*bounds, width, height)

#     # Rasterize the GeoDataFrame
#     shapes = ((geom, 1) for geom in gdf.geometry)
#     raster = rasterize(
#         shapes,
#         out_shape=(height, width),
#         transform=transform,
#         fill=0,
#         all_touched=True,
#         dtype='float32'
#     )

#     # Convert zero values to NaN to make them transparent
#     raster = np.where(raster == 0, np.nan, raster)

#     # Normalize data for colormap scaling, ignoring NaNs
#     norm = Normalize(vmin=np.nanmin(raster), vmax=np.nanmax(raster))
#     colormap_instance = ScalarMappable(norm=norm, cmap=colormap)
    
#     # Apply colormap, converting NaNs to transparent
#     colored_raster = colormap_instance.to_rgba(raster)
#     colored_raster[..., 3] = np.where(np.isnan(raster), 0, colored_raster[..., 3])  # Set NaN values as fully transparent

#     # Save the rasterized layer as PNG with transparency
#     output_path = f'Raster/data/{layer_name}.png'
#     plt.imsave(output_path, colored_raster, format='png', origin='upper')

#     # Define the correct bounding box for the ImageOverlay
#     image_bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]

#     # Add the color-mapped image as an ImageOverlay with correct bounds
#     ImageOverlay(
#         name=layer_name,
#         image=output_path,
#         bounds=image_bounds,
#         opacity=opacity
#     ).add_to(map_obj)

# # Example usage with your main Folium map
# m = folium.Map(location=[14.6349, -86.8287], zoom_start=7, tiles='CartoDB Positron')

# # Load your road shapefiles
# drivable_roads = gpd.read_file('Raster/honduras_roads_drive_service/honduras_roads_drive_service.shp')
# walkable_roads = gpd.read_file('Raster/honduras_roads_walk/honduras_roads_walk.shp')


In [88]:
def add_flood_layer(gdf, map_obj, layer_name, color_map='Blues', opacity=0.5, num_intervals=5):
    # Ensure that 'sum' is a numeric type
    gdf['sum'] = pd.to_numeric(gdf['sum'], errors='coerce')
    
    # Calculate the minimum and maximum of the 'sum' attribute
    min_sum = gdf['sum'].min()
    max_sum = gdf['sum'].max()
    
    # Create equal intervals
    intervals = np.linspace(min_sum, max_sum, num_intervals + 1)
    
    # Create a color map using the updated Matplotlib API
    cmap = plt.colormaps[color_map]  # Use the new method for getting the colormap
    colors = [cmap(i / num_intervals) for i in range(num_intervals)]
    
    # Convert RGBA to hex
    hex_colors = ['#' + ''.join(f'{int(c * 255):02x}' for c in color[:3]) for color in colors]
    
    # Function to determine the color based on the 'sum' attribute
    def get_color(feature):
        value = feature['properties']['sum']
        if pd.isna(value):
            return 'transparent'  # or some other fallback color
        for i in range(len(intervals) - 1):
            if intervals[i] <= value < intervals[i + 1]:
                return hex_colors[i]
        return hex_colors[-1]  # in case it's equal to the max
    
    # Add the GeoJson layer to the map
    folium.GeoJson(
        gdf,
        name=layer_name,
        style_function=lambda feature: {
            'color': 'none',  # No stroke
            'weight': 0.1,
            'fillColor': get_color(feature),  # Use the color determined by the function
            'fillOpacity': opacity
        }
    ).add_to(map_obj)


In [129]:

aoi_gdf = gpd.read_file('AOI/Edesur.shp')
police = gpd.read_file('Raster/police')
health = gpd.read_file('Raster/health')
schools = gpd.read_file('Raster/schools')


# Create a base map centered on the area of interest with Positron basemap
m = folium.Map(location=[18.4161, -70.1058], zoom_start=10, tiles='CartoDB Positron')


folders_zonal = [
    'Raster/00_Zonal_Shapefiles'
]
folders_existing = [
    'Raster/SCNO101/Existing',
    'Raster/SCNO102/Existing',
    'Raster/SCNO103/Existing'
]

folders_SCNO101 = [
    'Raster/SCNO101/Transformers',
    'Raster/SCNO101/Planned'
]

folders_SCNO102 = [
    'Raster/SCNO102/Transformers',
    'Raster/SCNO102/Planned'
]

folders_SCNO103 = [
    'Raster/SCNO103/Transformers',
    'Raster/SCNO103/Planned'
]
def add_aoi_layer(gdf, map_obj, layer_name, color='blue', opacity=0.5):
    folium.GeoJson(
        gdf,
        name=layer_name,
        style_function=lambda feature: {
            'color': 'black',
            'weight': 0.5,
            'fillColor': 'none',
            'fillOpacity': opacity
        }
    ).add_to(map_obj)

def add_gdf_layer(gdf, map_obj, layer_name, color='blue', opacity=0.5):
    folium.GeoJson(
        gdf,
        name=layer_name,
        style_function=lambda feature: {
            'color': color,
            'weight': 1,
            'fillColor': color,
            'fillOpacity': opacity
        }
    ).add_to(map_obj)



def add_point_layer(gdf, map_obj, layer_name, color='blue', radius=5):
    for _, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            popup=layer_name
        ).add_to(map_obj)



# Function to load, check CRS, and add layers from folders
def load_and_add_layers(folders, map_obj, color, opacity, layer_prefix):
    for folder in folders:
        shapefiles = glob.glob(os.path.join(folder, '*.shp'))
        for file_path in shapefiles:
            try:
                # Read each shapefile as a GeoDataFrame
                gdf = gpd.read_file(file_path)
                # Check if the GeoDataFrame is not empty
                if gdf.empty:
                    print(f"Warning: {file_path} is empty and will not be added.")
                    continue
                
                # Ensure the CRS is WGS84 (EPSG:4326)
                if gdf.crs and gdf.crs.to_epsg() != 4326:
                    gdf = gdf.to_crs(epsg=4326)
                
                # Extract layer name
                layer_name = f"{layer_prefix} - {os.path.basename(file_path).replace('.shp', '')}"
                
                # Add the GeoDataFrame to the map
                add_gdf_layer(gdf, map_obj, layer_name, color=color, opacity=opacity)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

def load_and_add_flood_layers(folders, map_obj, color, opacity, layer_prefix):
    for folder in folders:
        shapefiles = glob.glob(os.path.join(folder, '*.shp'))
        for file_path in shapefiles:
            try:
                # Read each shapefile as a GeoDataFrame
                gdf = gpd.read_file(file_path)
                # Check if the GeoDataFrame is not empty
                if gdf.empty:
                    print(f"Warning: {file_path} is empty and will not be added.")
                    continue
                
                # Ensure the CRS is WGS84 (EPSG:4326)
                if gdf.crs and gdf.crs.to_epsg() != 4326:
                    gdf = gdf.to_crs(epsg=4326)
                
                # Extract layer name
                layer_name = f"{layer_prefix} - {os.path.basename(file_path).replace('.shp', '')}"
                
                # Add the GeoDataFrame to the map
                add_flood_layer(gdf, map_obj, layer_name, color_map=color, opacity=opacity, num_intervals=5)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")

load_and_add_flood_layers(folders_zonal, m, color='Blues', opacity=0.3, layer_prefix='Flood')
load_and_add_layers(folders_existing, m, color='grey', opacity=0.3, layer_prefix='Existing')
load_and_add_layers(folders_SCNO101, m, color='magenta', opacity=0.3, layer_prefix='SCNO101')
load_and_add_layers(folders_SCNO102, m, color='orange', opacity=0.3, layer_prefix='SCNO102')
load_and_add_layers(folders_SCNO103, m, color='red', opacity=0.3, layer_prefix='SCNO103')
add_colored_tiff_layer(m, tiff_path, layer_name='Economic Activity',colormap='Spectral',vectorize=False)
add_colored_tiff_layer(m, tiff_path ='Raster/edesur_population.tif', layer_name='Population',colormap='YlOrRd',vectorize=False)
add_colored_tiff_layer(m, tiff_path ='Raster/edesur_slope.tif', layer_name='Slope',colormap='Purples',vectorize=False)
add_colored_tiff_layer(m, tiff_path ='Raster/edesur_lc_burn.tif', layer_name='Wildfire',colormap='BrBG',vectorize=False)
add_point_layer(police, m, 'Police', color='red', radius=1)
add_point_layer(health, m, 'Health', color='magenta', radius=1)
add_point_layer(schools, m, 'Schools', color='green', radius=1)
add_aoi_layer(aoi_gdf, m, 'Area of Interest', color= "white")


# Add Layer Control
folium.LayerControl().add_to(m)

m.keep_layers_off=True

# Display the map
m


# Save the map as an HTML file if needed
m.save('Output/index.html')
print("Map saved as index.html")


Map saved as output_map.html
